# PDEs and their solutions

General partial differential equation I would like to solve:
$$
G\Big( \vec{x}, \; f(\vec{x}), \;  \partial_{i}f(\vec{x}), \; \partial_{i}\partial_{j}f(\vec{x}) \Big) = 0
$$
for $\vec{x} \in \Omega$ (i.e. some $n$-dimentional domain).

In order for this to have a solution, the following must be true (the *Cauchy–Kowalevski theorem*):

1.  We should be able to solve for the highest derivative. That is, there should exist an analytic $F$ so that 
$$
\partial_k\partial_k f(\vec{x})=F\Big(\vec{x}, f(\vec{x}), \partial_i f(\vec{x}), 
\left\{ \partial_i \partial_j f(\vec{x}) \right\}_{ij \neq kk} \Big)
$$

2. We should know:
$$
f(\vec{x})\Big|_{x_{k}=x_{k}^{(0)}} = L_{0}\Big(\left\{ x_i \right\}_{i \neq k}\Big)
\\
\partial_k f(\vec{x})\Big|_{x_{k}=x_{k}^{(0)}} = L_{1}\Big(\left\{ x_i \right\}_{i \neq k}\Big) 
$$

## Usual PDE problems

In general proving that the given conditions give unique well define solution is non-trivial. However, usually, PDEs are given with some *boundary conditions* of the form
$$
H\Big(\vec{x},f(\vec{x}),\partial_i f(\vec{x}) \Big)\Big|_{\vec{x} \in {\bf S} } =0 \;,
$$
with ${\bf S}$ the boundary of the region in which we look for a solution.

In [1]:
import os

os.chdir('Optimizers/Stochastic-Gradient-Descent/python/')
import SGD as SGD 
os.chdir('../../../')

import numpy as np

import matplotlib
matplotlib.use('nbAgg')
import matplotlib.pyplot as plt

import time

# Example

Let's try solve the diffusion equation
$$
\dfrac{\partial u(x,t)}{\partial t} - \alpha^2 \dfrac{\partial^2 u(x,t)}{\partial x^2}= 0
$$
for $x \in [0,1]$ and $t>0$. In order to find a solution, we need
1. initial condition that fix  $u(x,0)=\sin( \pi \ x)$
2. boundary conditions that fix $u(0,t)=u(1,t)=0$


Notice that the solution is  
$$
u(x,t) =  \sin( \pi \ x) \ e^{-(\pi \ \alpha )^2 t}
$$

For simplicity, we'll set $a=\dfrac{1}{2}$.


In [2]:
import os

os.chdir('Optimizers/Stochastic-Gradient-Descent/python/')
import SGD as SGD 
os.chdir('../../../')

import numpy as np

import matplotlib
matplotlib.use('nbAgg')
import matplotlib.pyplot as plt

import time

As a sanity check, we choose
$$
u=w_0 \ \sin(w_1 \ x) e^{ w_2 \ t}
$$

the solution is then given for
$$
w_0 = \pm 1 \\
w_1 = \pm \pi \\
w_2 = -\left(\dfrac{\pi}{2}\right)^2
$$

### Note that we have to choose a region for t. So let's will choose $0<t<2$ 

In [3]:
class modelFunc(SGD.modelBase):
    def __init__(self,dimensions,w0,h=1e-3):
        SGD.modelBase.__init__(self,dimensions,w0)

        self.ds_ldx_i=0#derivative of the l^th wrt the i^th input
        self.d2s_ldx_ij=0#second derivative of the l^th wrt the i^th and j^th inputs
        self.d2s_ldx_ii=0#second derivative of the l^th wrt the i^th input
        
        self.h=h
        

    def __call__(self):
        self.signal[0]=self.w[0]*np.sin(self.w[1]*self.input[0]) * np.exp(self.w[2]*self.input[1])
    
    
    
    def derivative_ij(self,l,i,j):
        heff1=self.h+np.abs(self.input[i])*self.h
        heff2=self.h+np.abs(self.input[j])*self.h
        
        self.input[i]+=heff1
        self.input[j]+=heff2
        self()
        f_ff=self.signal[l]
        
        self.input[i]-=2*heff1
        self.input[j]-=2*heff2
        self()
        f_bb=self.signal[l]
        
        self.input[i]+=2*heff1
        self()
        f_fb=self.signal[l]
        
        self.input[i]-=2*heff1
        self.input[j]+=2*heff2
        self()
        f_bf=self.signal[l]
        
        self.input[i]+=heff1
        self.input[j]-=heff2
        
        
        self.d2s_ldx_ij = (f_ff+f_bb-f_fb-f_bf)/(4*heff1*heff2)
        
        
    def derivative_ii(self,l,i):
        heff=self.h+np.abs(self.input[i])*self.h

        self.input[i]-=heff
        self()
        f_b=self.signal[l]
        
        self.input[i]+=2*heff
        self()
        f_f=self.signal[l]
        
        self.input[i]-=heff
        self()
        f_0=self.signal[l]

        self.d2s_ldx_ii = (f_f+f_b-2*f_0)/(heff**2)
    
    
    def derivative_i(self,l,i):
        heff=self.h+np.abs(self.input[i])*self.h
        
        self.input[i]-=heff
        self()
        f0=self.signal[l]

        self.input[i]+=2*heff
        self()
        f1=self.signal[l]
        
        self.input[i]-=heff

        self.ds_ldx_i = (f1-f0)/(2*heff)

# The boundary conditions

Boundary conditions is given as in the class below. 

It would be convinient to define a function that returns a random point inside the boundary, which will be used to generate points that will be used to train the model.


I also think that the contribution of the boundary conditions to the loss should be included here (as in the ODE case). 

In contrast to the ```Boundary``` class of the ODE example, we choose the points to be generated "on the fly" and not binge passed in the initializations, because in general this may be dificult to do (teh boundary condition can be very complicated). 

In [4]:
class Boundary:
    def __init__(self,model):
        self.model=model
        
        self.NConditions=3
        self.insidePoint=[0 for _ in range(self.model.dimensions[0])]# this will hold the inputs inside the boundary
        self.boundaryPoints=[[0 for _1 in range(self.model.dimensions[0])] for _2 in range(self.NConditions)]# this will hold the inputs on the boundary for all conditions       
        self.boundary_diff=[[0 for _1 in range(self.model.dimensions[1])] for _2 in range(self.NConditions)]# this will hold the difference of the boundary conditions for all signals

    #get a random point in the region of interest
    def randomPoint(self):
        x=np.random.rand()
        t=np.random.rand()*2
        
        self.insidePoint=[x,t]
    
        
    def randomBoundaryPoints(self):
        '''get a  point on the boundary for each boundary condition'''
        
        self.randomPoint()
        x1=self.insidePoint[:]
        x1[0]=0

        self.randomPoint()
        x2=self.insidePoint[:]
        x2[0]=1

        self.randomPoint()
        x3=self.insidePoint[:]
        x3[1]=0
        
        self.boundaryPoints=[x1,x2,x3]

 
    def randomBoundaryConditions(self):
        tmp_p=self.model.input[:]
        
        self.randomBoundaryPoints()
        
        
        for _c in range(self.NConditions):
            #get difference for the all conditions
            self.model.setInput(self.boundaryPoints[_c])
            self.model()
            self.boundary_diff[_c][0]=self.model.signal[0]

        self.boundary_diff[2][0]+=-np.sin(np.pi*self.boundaryPoints[2][0])
        
        self.model.setInput(tmp_p)
        
    def randomBoundaryLoss(self):
        '''average loss of the boundary conditions'''
        self.randomBoundaryConditions()
        avgBoundaryLoss=0
        for _c in range(self.NConditions):
            for l in range(self.model.dimensions[1]):
                avgBoundaryLoss+=self.boundary_diff[_c][l]**2
        
        return avgBoundaryLoss/(1.*(self.NConditions*self.model.dimensions[1]))
        


# The PDE

Just define a class that holds everything that is relevant. Basically, you want have a place that can give you the loss easily.

Overload the ```__call__``` function to return $\rm lhs-rhs$. Define the loss and its derivarative over 
$\{\bf{w}\}$. 

In [5]:
class DifferentialEquation:
    def __init__(self,Model,Boundary,h=1e-2):
        self.RHS=[lambda x:0]

        self.model=Model
        self.boundary=Boundary
        
        self.DE=[0 for _ in range(self.model.dimensions[1])]#this should be LHS-RHS, which we want to be 0 (see definition of __call__)
        self.h=h
        self.dQdw=0
        
    def __call__(self):
        self.model.derivative_i(0,1)
        self.model.derivative_ii(0,0)
        
        
        dudt = self.model.ds_ldx_i
        d2udx2 = self.model.d2s_ldx_ii
        alpha=0.5
        
        self.DE[0]= dudt - alpha**2 * d2udx2 - self.RHS[0](self.model.input)
    
    
    def randomDataPoint(self):
        self.boundary.randomPoint()
        self.model.setInput(self.boundary.insidePoint)

        
    def averageLoss(self):
        '''get the average total loss'''
        
        avrgLoss=0
        #loss from the PDE at the random point 
        self()
        for l in range(self.model.dimensions[1]):
            avrgLoss+=(self.DE[l]**2)/(1.*self.model.dimensions[1])
        
        #loss for the boundary conditions
        avrgLoss+=self.boundary.randomBoundaryLoss()
        
        return avrgLoss/(2.)
        

    def grad(self,i):
        '''Get the gradient of the averge loss wrt w[i] at current point + initial conditon'''
        '''we should find a way to do it analytically (at least the dQds part)'''
        
        heff=self.h*np.abs(self.model.w[i])+self.h

        self.model.w[i]-=heff
        Q0=self.averageLoss()

        self.model.w[i]+=2*heff
        Q1=self.averageLoss()

        self.dQdw=(Q1-Q0)/(2.*heff)

        self.model.w[i]-=heff


In [6]:
model=modelFunc([2,1], np.random.rand(3)*2-1 )
S=Boundary(model)
PDE=DifferentialEquation(model,S)

In [7]:
# sgd=SGD.VanillaSGD(PDE,alpha=1e-2)
# sgd=SGD.RMSpropSGD(PDE,gamma=1-1e-2,epsilon=1e-5,alpha=1e-1)
# sgd=SGD.AdaDeltaSGD(PDE,gamma=0.99,epsilon=1e-5,alpha=1)
# sgd=SGD.AdamSGD(PDE,beta_m=0.9,beta_v=0.999,epsilon=1e-8,alpha=1e-1)
# sgd=SGD.AdaMaxSGD(PDE,beta_m=0.9,beta_v=0.999,epsilon=1e-8,alpha=1e-1)
sgd=SGD.NAdamSGD(PDE,beta_m=0.9,beta_v=0.999,epsilon=1e-8,alpha=1e-1)

In [8]:
time0=time.time()

sgd.run(abs_tol=1e-3, rel_tol=1e-3, step_break=5000,max_step=50000)

time.time()-time0,len(sgd.steps)

(4.12401008605957, 5788)

In [9]:
#As you can see
print('I got w=',model.w)
print('I expect w= [','+-1',',+-',np.pi,',',-(np.pi/2)**2,']')

I got w= [-1.00033704 -3.14248718 -2.47484671]
I expect w= [ +-1 ,+- 3.141592653589793 , -2.4674011002723395 ]


The loss for a lot of random points is small!

In [10]:
maxLoss=0
for i in range(10000):
    PDE.randomDataPoint()
    loss=PDE.averageLoss()
    if loss>maxLoss:
        maxLoss=loss
maxLoss

1.81244301466782e-05